In [1]:
import sqlite3 as sq
from pprint import pprint
import pandas as pd

In [2]:
def make_query(query):
    with sq.connect('decanat.db') as con:
        result = con.execute(query)
        names = [description[0] for description in result.description]
        result = pd.DataFrame(result, columns=names)
    display(result)

# Запросы на выборку

#### 1. Вывести список всех студентов, зачисленных на факультет, с указанием номера группы.

1. Факультет Информационных Технологий
2. Факультет Робототехники и Мехатроники
3. Факультет Энергетики и Энергомашиностроения
4. Факультет Нанотехнологий и Материаловедения
5. Факультет Авиационной и Космической Техники
6. Факультет Прикладной Математики и Информатики
7. Факультет Биотехнологии и Биоинженерии
8. Факультет Электроники и Наноэлектроники
9. Факультет Гражданского и Промышленного Строительства
10. Факультет Транспортных Технологий и Систем

In [3]:
#Укажите номера факультетов, по которым показать студентов
choose_faculty_id = (1,10)

query_1_1 = f"""
            --sql
            with table2 as (with table1 as (select faculty.id as f_id, faculty.faculty_name, departments.id as d_id, departments.depatment_name
            from faculty
            left join departments
            on departments.id_faculty == faculty.id
            where faculty.id in {choose_faculty_id})
            
            select table1.f_id, table1.faculty_name, table1.depatment_name, specialties.id as s_id, specialties.specialty_name
            from table1
            left join specialties
            on specialties.id_department = table1.d_id)
            
            
            select table2.faculty_name, students.surname, students.student_name, students.fathers_name, 
                students.group_num
            from table2
            left join students
            on table2.s_id = students.student_specialty
            
            
            
            
            --endsql    
        """
make_query(query_1_1)

,faculty_name,surname,student_name,fathers_name,group_num
0,Факультет Информационных Технологий,Абдуллаева,Виолетта,Симбирская,ИМТ-17
1,Факультет Информационных Технологий,Абузяров,Вавила,Никонов,ИБА-20
2,Факультет Информационных Технологий,Аверченко,Егор,Земляной,ИВТ-22
3,Факультет Информационных Технологий,Авксентьева,Аврора,Данькова,ИБА-9
4,Факультет Информационных Технологий,Арцыбашев,Куприян,Грин,ИБА-21
...,...,...,...,...,...
1990,Факультет Транспортных Технологий и Систем,Чиков,Арсений,Безбатько,ИМТ-1
1991,Факультет Транспортных Технологий и Систем,Чурова,Афина,Гагарина,ИС-1
1992,Факультет Транспортных Технологий и Систем,Шпанова,Теона,Аннинская,ИКТ-22
1993,Факультет Транспортных Технологий и Систем,Юдов,Алексей,Углев,ИВТ-16


#### 2. Вывести список всех специальностей факультета и количество студентов, обучающихся по каждой из них.

In [4]:
query_1_2 = f"""
            --sql
            with table2 as (with table1 as (                
                select faculty.id as f_id, departments.id as d_id 
            from faculty
            left join departments
            on departments.id_faculty = faculty.id)
            
            select table1.f_id, table1.d_id, specialties.id as s_id, specialties.specialty_name
           from table1
            left join specialties
           on specialties.id_department = table1.d_id)
           
           
           select s_id, table2.specialty_name, count(students.id) as num_students_on_specialty 
           from table2
           left join students
           on students.student_specialty = table2.s_id
           group by s_id
           order by count(students.id)  desc, s_id
            --endsql    
        """
make_query(query_1_2)

,s_id,specialty_name,num_students_on_specialty
0,47,Численные методы,138
1,2,Информатика и вычислительная техника,136
2,12,Беспилотные авиационные системы,134
3,56,Молекулярная биология и генетика,130
4,58,Медицинские приборы и системы,129
...,...,...,...
85,9,Когнитивные вычисления,94
86,7,Физика и информатика,92
87,48,Вычислительные системы и сети,92
88,61,Охрана окружающей среды,89


#### 3. Вывести список всех кафедр факультета и количество студентов, обучающихся на каждой кафедре.

In [5]:
query_1_3 = f"""
            --sql
            with table1 as (select departments.depatment_name, specialties.id as s_id
            from departments
            left join specialties
            on specialties.id_department = departments.id)
            
            select table1.depatment_name, count(students.id) as num_students_on_department
            from table1
            left join students
            on table1.s_id = students.student_specialty
            group by table1.depatment_name
            order by num_students_on_department desc
        """
make_query(query_1_3)

,depatment_name,num_students_on_department
0,Кафедра Биомедицинской Инженерии,361
1,Кафедра Прикладной Информатики,359
2,Кафедра Программной Инженерии,351
3,Кафедра Компьютерных Систем и Сетей,351
4,Кафедра Мехатроники и Автоматизации,350
5,Кафедра Космической Техники,349
6,Кафедра Морского и Речного Транспорта,344
7,Кафедра Гражданского Строительства,344
8,Кафедра Телекоммуникаций,343
9,Кафедра Вычислительной Математики,340


#### 4. Вывести список всех предметов и количество часов, выделенных на каждый предмет в учебном плане каждой специальности.

In [6]:
query_1_4 = f"""
            --sql
            with table1 as (select id_subject, id_specialtie, (num_lecture_hours + num_practical_hours + num_lab_hours) as num_hours
            from study_program
            group by id_subject, id_specialtie)
            
            select subjects.subject_name as subjects_name, specialties.specialty_name, table1.num_hours
            from table1
            left join subjects
            on subjects.id = table1.id_subject
            left join specialties
            on specialties.id = table1.id_specialtie
            order by num_hours desc
        """ 
make_query(query_1_4)

,subjects_name,specialty_name,num_hours
0,Элементы высшей математики,Теплоэнергетические установки,210
1,История,Промышленные сооружения,210
2,Элементы высшей математики,Машинное обучение и анализ данных,180
3,Иностранный язык,Численные методы,180
4,Психология общения,Интеллектуальные транспортные системы,180
5,Психология общения,Теплогидроизоляционные работы,180
6,Психология общения,Нормирование и контроль в строительстве,180
7,Облачные технологии,Микро- и наноэлектроника,180
8,Численные методы,Проектирование и эксплуатация воздушных судов,165
9,Иностранный язык,Информационные технологии,165


#### 5. Вывести список всех студентов, у которых есть неудовлетворительные оценки (меньше 4) по любому предмету.

In [7]:
query_1_5 = f"""
            --sql
            select surname, student_name, fathers_name, mark
            from study_card
            left join students
            on students.id = study_card.id_student
            where mark < 4
            
        """ 
make_query(query_1_5)

,surname,student_name,fathers_name,mark
0,Маркианов,Якун,Маликов,2
1,Соболь,Антонин,Зотьев,2
2,Прокопова,Мавлюда,Солопова,3
3,Сенчина,Инесса,Нетёсова,3
4,Епинетова,Лира,Ненарокова,2
...,...,...,...,...
5010,Развожаева,Милада,Графинина,2
5011,Суворов,Никон,Житинский,2
5012,Кисин,Филипп,Мурманцев,3
5013,Гладилина,Физалия,Китова,2


#### 6. Вывести список всех предметов, которые изучают студенты первого курса.

In [8]:
query_1_6 = f"""
            --sql
            SELECT  subjects.subject_name, strftime('%Y',students.entry_date) - 2021 as num_of_course
FROM study_card AS s_card
LEFT JOIN students 
  ON students.id = s_card.id_student
  left join subjects
  on subjects.id = s_card.id_subject
  where strftime('%Y',students.entry_date) = '2022'
  group by subjects.subject_name
  
        """ 
make_query(query_1_6)

,subject_name,num_of_course
0,Иностранный язык,1
1,История,1
2,Облачные технологии,1
3,Операционные системы и среды,1
4,Основы алгоритмизации и программирования,1
5,Основы кибернетики и робототехники,1
6,Психология общения,1
7,Физкультура,1
8,Численные методы,1
9,Элементы высшей математики,1


#### 7. Вывести список всех студентов, которые сдают курсовую работу в этом семестре.

In [9]:
query_1_7 = f"""
            --sql
            select students.surname,	students.student_name,	students.fathers_name, subject_methods.subject_method_name
            from study_card
            left join students
            on students.id = study_card.id_student
            left join subject_methods
            on subject_methods.id = study_card.id_subject
            where subject_methods.subject_method_name = 'Курсовая работа'
            
        """ 
make_query(query_1_7)

,surname,student_name,fathers_name,subject_method_name
0,Сударушкин,Валерий,Егорьев,Курсовая работа
1,Гаврилов,Григорий,Смольников,Курсовая работа
2,Офросимова,Физалия,Слепухина,Курсовая работа
3,Каткова,Элеонора,Солодникова,Курсовая работа
4,Алешковский,Егор,Федосов,Курсовая работа
...,...,...,...,...
941,Филатьева,Стефания,Арабова,Курсовая работа
942,Литаврина,Жоржина,Аристова,Курсовая работа
943,Садур,Богдан,Иванкин,Курсовая работа
944,Королева,Зарема,Поляновская,Курсовая работа


#### 8. Вывести список всех абитуриентов, зачисленных на специальность "Информатика и вычислительная техника".

In [10]:
query_1_8 = f"""
            --sql
            select specialties.specialty_name, students.surname, students.student_name, students.fathers_name
            from students
            left join specialties
            on specialties.id = students.student_specialty
            where specialties.specialty_name = 'Информатика и вычислительная техника'
        """ 
make_query(query_1_8)

,specialty_name,surname,student_name,fathers_name
0,Информатика и вычислительная техника,Симеонов,Магистриан,Блок
1,Информатика и вычислительная техника,Горбачёв,Эдуард,Ягужинский
2,Информатика и вычислительная техника,Пригожин,Эдуард,Милов
3,Информатика и вычислительная техника,Арепьева,Сильвия,Иовлева
4,Информатика и вычислительная техника,Невзоров,Константин,Лавров
...,...,...,...,...
131,Информатика и вычислительная техника,Платонова,Римма,Турова
132,Информатика и вычислительная техника,Луппова,Анастасия,Водденикова
133,Информатика и вычислительная техника,Насонова,Симона,Рогова
134,Информатика и вычислительная техника,Доримедонтова,Снежана,Довлатова


#### 9. Вывести список всех предметов, которые изучают студенты группы 101.

In [11]:
query_1_9 = f"""
            --sql
            with table1 as (select id_subject
            from students
            left join study_card
            on students.id = study_card.id_student
            where students.group_num = '101'
            group by id_subject)
            
            select subject_name
            from table1
            left join subjects
            on subjects.id = table1.id_subject
            where subject_name is not NULL
            
            
        """ 
make_query(query_1_9)

,subject_name
0,Операционные системы и среды
1,Основы алгоритмизации и программирования
2,Элементы высшей математики
3,Основы кибернетики и робототехники
4,Численные методы
5,Физкультура
6,Иностранный язык
7,Психология общения
8,Облачные технологии
9,История


#### 10. Вывести список студентов и их оценки за все предметы на специальности "Программная инженерия"

In [12]:
query_1_10 = f"""
            --sql
            select specialties.specialty_name, students.surname, students.student_name, subjects.subject_name, study_card.mark
            from study_card
            left join students
            on students.id = study_card.id_student
            left join specialties
            on specialties.id = study_card.id_specialty
            left join subjects
            on subjects.id = study_card.id_subject
            where specialties.specialty_name = 'Программная инженерия'
            
        """ 
make_query(query_1_10)

,specialty_name,surname,student_name,subject_name,mark
0,Программная инженерия,Фирсова,Елена,Основы алгоритмизации и программирования,4
1,Программная инженерия,Лутьянов,Владислав,Психология общения,4
2,Программная инженерия,Федина,Чеслава,Основы алгоритмизации и программирования,3
3,Программная инженерия,Лавров,Эдуард,Элементы высшей математики,4
4,Программная инженерия,Пастухов,Евграф,Психология общения,3
...,...,...,...,...,...
110,Программная инженерия,Смехов,Ростислав,Физкультура,2
111,Программная инженерия,Нестерова,Ульяна,Психология общения,2
112,Программная инженерия,Грин,Савва,Облачные технологии,2
113,Программная инженерия,Киктенко,Ефим,История,5


# Запросы на обновление данных из БД

#### 1.Обновление названия факультета с id=1 на "Новый факультет"

In [13]:
with sq.connect('decanat.db') as con:
    query_2_1 = f"""
                --sql   
                update faculty
                set faculty_name = 'Новый факультет'
                where id = 1
                --endsql
            """ 
    con.execute(query_2_1)
    


query_2_1_ = f"""
            --sql   
            select * 
            from faculty
            limit 3;
            --endsql
        """ 
make_query(query_2_1_)

,id,faculty_name
0,1,Новый факультет
1,2,Факультет Робототехники и Мехатроники
2,3,Факультет Энергетики и Энергомашиностроения


#### 2. Обновление названия кафедры с id=2 на "Новая кафедра"

In [14]:
with sq.connect('decanat.db') as con:
    query_2_2 = f"""
                --sql   
                update departments
                set depatment_name = 'Новая кафедра'
                where id = 2
                --endsql
            """ 
    con.execute(query_2_2)
    


query_2_2_ = f"""
            --sql   
            select * 
            from departments
            limit 5;
            --endsql
        """ 
make_query(query_2_2_)

,id,depatment_name,id_faculty
0,1,Кафедра Компьютерных Систем и Сетей,1
1,2,Новая кафедра,1
2,3,Искусственного Интеллекта и Машинного Обучения,1
3,4,Кафедра Робототехники,2
4,5,Кафедра Мехатроники и Автоматизации,2


#### 3. Обновление названия специальности с id=3 на "Новая специальность"

In [15]:
with sq.connect('decanat.db') as con:
    query_2_3 = f"""
                --sql   
                update specialties
                set specialty_name = 'Новая специальность'
                where id = 3
                --endsql
            """ 
    con.execute(query_2_3)
    


query_2_3_ = f"""
            --sql   
            select * 
            from specialties
            limit 5;
            --endsql
        """ 
make_query(query_2_3_)

,id,specialty_name,id_department
0,1,Архитектура вычислительных систем,1
1,2,Информатика и вычислительная техника,1
2,3,Новая специальность,1
3,4,Математика и информатика,2
4,5,Информационные технологии,2


#### 4. Обновление названия предмета с id=4 на "Новый предмет"

In [16]:
with sq.connect('decanat.db') as con:
    query_2_4 = f"""
                --sql   
                update subjects
                set subject_name = 'Новый предмет'
                where id = 4
                --endsql
            """ 
    con.execute(query_2_4)
    


query_2_4_ = f"""
            --sql   
            select * 
            from subjects
            limit 5;
            --endsql
        """ 
make_query(query_2_4_)

,id,subject_name
0,0,Информационные технологии
1,1,Операционные системы и среды
2,2,Основы алгоритмизации и программирования
3,3,Элементы высшей математики
4,4,Новый предмет


#### 5. Обновление названия формы сдачи предмета с id=5 на "Новая форма сдачи"

In [17]:
with sq.connect('decanat.db') as con:
    query_2_5 = f"""
                --sql   
                update subject_methods
                set subject_method_name = 'Новая форма сдачи'
                where id = 5
                --endsql
            """ 
    con.execute(query_2_5)
    


query_2_5_ = f"""
            --sql   
            select * 
            from subject_methods
            --endsql
        """ 
make_query(query_2_5_)

,id,subject_method_name
0,0,Дифференцированный зачет
1,1,Экзамен
2,2,Зачет
3,3,Недифференцированный зачет
4,4,Курсовая работа
5,5,Новая форма сдачи
6,6,Контрольная работа


#### 6. Обновление количества лекционных часов на 30 для учебного плана с id=6

In [18]:
with sq.connect('decanat.db') as con:
    query_2_6 = f"""
                --sql   
                update study_program
                set num_lecture_hours = 30
                where id = 6
                --endsql
            """ 
    con.execute(query_2_5)
    


query_2_6_ = f"""
            --sql   
            select * 
            from study_program
            limit 10;
            --endsql
        """ 
make_query(query_2_6_)

,id,id_specialtie,id_subject,id_subject_method,num_lecture_hours,num_practical_hours,num_lab_hours,final_task
0,0,48,1,0,30,30,0,1
1,1,85,2,0,60,0,0,0
2,2,36,3,3,30,0,15,0
3,3,24,4,2,30,30,30,1
4,4,20,5,1,60,30,0,0
5,5,19,6,1,30,30,30,1
6,6,18,7,2,30,0,0,0
7,7,5,8,0,60,60,0,0
8,8,52,9,2,30,30,0,0
9,9,52,9,2,30,30,0,0


#### 7. Обновление количества лекционных часов у предмета "Математика" на учебном плане специальности "Математика и информатика"

In [19]:
print('До')
query_2_7_ = f"""
            --sql   
            select * 
            from study_program
            where id_specialtie = 4 and id_subject = 12
            --endsql
        """ 
make_query(query_2_7_)

with sq.connect('decanat.db') as con:
    query_2_7 = f"""
                --sql   
                update study_program
                set num_lecture_hours = 60
                where id_specialtie = 4 and id_subject = 12
                --endsql
            """ 
    con.execute(query_2_7)
    

print('После')
make_query(query_2_7_)

До


,id,id_specialtie,id_subject,id_subject_method,num_lecture_hours,num_practical_hours,num_lab_hours,final_task
0,51,4,12,0,30,30,30,0


После


,id,id_specialtie,id_subject,id_subject_method,num_lecture_hours,num_practical_hours,num_lab_hours,final_task
0,51,4,12,0,60,30,30,0


#### 8. Обновление количества лабораторных часов и формы сдачи предмета у специальности "Программная инженерия" для предмета "Программирование"

In [20]:
print('До')
query_2_8_ = f"""
            --sql   
            select * 
            from study_program
            where id_specialtie = 3 and id_subject = 13
            --endsql
        """ 
make_query(query_2_8_)

with sq.connect('decanat.db') as con:
    query_2_8 = f"""
                --sql   
                update study_program
                set num_lab_hours = 120
                where id_specialtie = 3 and id_subject = 13
                --endsql
            """ 
    con.execute(query_2_8)
    

print('После')
make_query(query_2_8_)

До


,id,id_specialtie,id_subject,id_subject_method,num_lecture_hours,num_practical_hours,num_lab_hours,final_task
0,52,3,13,0,30,30,30,0


После


,id,id_specialtie,id_subject,id_subject_method,num_lecture_hours,num_practical_hours,num_lab_hours,final_task
0,52,3,13,0,30,30,120,0


#### 9. Обновление количества лекционных и практических часов у предмета "Информатика" на учебном плане кафедры "Информационных технологий"

In [21]:
print('До')
query_2_9_ = f"""
            --sql   
            select * 
            from study_program
            where id_specialtie = 5 and id_subject = 14
            --endsql
        """ 
make_query(query_2_9_)

with sq.connect('decanat.db') as con:
    query_2_9 = f"""
                --sql   
                update study_program
                set num_lab_hours = 120
                where id_specialtie = 5 and id_subject = 14
                --endsql
            """ 
    con.execute(query_2_9)
    

print('После')
make_query(query_2_9_)

До


,id,id_specialtie,id_subject,id_subject_method,num_lecture_hours,num_practical_hours,num_lab_hours,final_task
0,53,5,14,0,30,30,30,0


После


,id,id_specialtie,id_subject,id_subject_method,num_lecture_hours,num_practical_hours,num_lab_hours,final_task
0,53,5,14,0,30,30,120,0


#### 10. Обновить кол-во лекционных часов для всех предметов, где количество лекционных часов больше 30

In [22]:
with sq.connect('decanat.db') as con:
    query_2_10 = f"""
                --sql   
                update study_program
                set num_lecture_hours = 45
                where num_lecture_hours > 30
                --endsql
            """ 
    con.execute(query_2_10)
    
query_2_10_ = f"""
            --sql   
            select * 
            from study_program
            where num_lecture_hours > 30
            limit 7;
            --endsql
        """ 
make_query(query_2_10_)

,id,id_specialtie,id_subject,id_subject_method,num_lecture_hours,num_practical_hours,num_lab_hours,final_task
0,1,85,2,0,45,0,0,0
1,4,20,5,1,45,30,0,0
2,7,5,8,0,45,60,0,0
3,11,67,3,3,45,60,30,0
4,12,5,7,1,45,90,30,0
5,13,29,10,1,45,30,45,0
6,15,65,3,2,45,45,0,0


#### 11. Обновить фамилию и имя абитуриента по его идентификатору

In [23]:
with sq.connect('decanat.db') as con:
    query_2_11 = f"""
                --sql   
                update students
                set surname = 'Новая фамилия', student_name = 'Новое имя'
                where id = 1
                --endsql
            """ 
    con.execute(query_2_11)
    

query_2_11_ = f"""
            --sql   
            select * 
            from students
            limit 7;
            --endsql
        """ 
make_query(query_2_11_)

,id,surname,student_name,fathers_name,gender,date_birth,address,phone,email,entry_date,student_specialty,group_num
0,1,Новая фамилия,Новое имя,Кисина,Ж,2001-06-21,Аллея Причальная 673,+7-(999)-709-59-62,trembl2074@yahoo.com,2019-07-10,86,ИС-22
1,2,Остратов,Валентин,Карпенков,М,2000-04-10,Аллея Кулаков 1251,+7-(914)-520-84-18,chad1946@yahoo.com,2021-02-17,85,ИВТ-14
2,3,Кириллова,Мирослава,Медкова,Ж,2005-01-16,ул. Николоворобинский Б. 234,+7-(922)-070-01-29,aka2016@live.com,2022-03-09,47,ИВТ-4
3,4,Филимонихин,Мирон,Пригожин,М,2000-01-17,ул. Капельская 167,+7-(909)-944-29-19,hurricane1985@yandex.com,2022-12-17,70,ИВТ-24
4,5,Обухова,Ея,Фалалеева,Ж,2000-12-03,Аллея Рыбников 973,+7-(960)-069-71-18,discovered1910@protonmail.com,2022-07-04,60,ИКТ-8
5,6,Евстафьев,Герман,Кабанов,М,2008-02-20,ул. Хохловка Нижн. 1049,+7-(933)-870-70-95,subaru1911@outlook.com,2022-12-10,39,ИМТ-19
6,7,Пономарева,Ярослава,Картофельникова,Ж,2000-05-03,ул. Пантелеевская 633,+7-(935)-325-72-23,foundations2025@yandex.com,2018-02-23,86,ИМТ-8


#### 12. Обновить название кафедры для всех специальностей, где кафедра имеет id = 1

In [24]:
with sq.connect('decanat.db') as con:
    query_2_12 = f"""
                --sql   
                update departments
                set depatment_name = 'Новая кафедра'
                where id = 1
                --endsql
            """ 
    con.execute(query_2_12)
    

query_2_12_ = f"""
            --sql   
            select * 
            from departments
            limit 7;
            --endsql
        """ 
make_query(query_2_12_)

,id,depatment_name,id_faculty
0,1,Новая кафедра,1
1,2,Новая кафедра,1
2,3,Искусственного Интеллекта и Машинного Обучения,1
3,4,Кафедра Робототехники,2
4,5,Кафедра Мехатроники и Автоматизации,2
5,6,Кафедра Интеллектуальных Транспортных Систем,2
6,7,Кафедра Теплоэнергетики,3


#### 13. Обновить оценку по определенному предмету и форме сдачи для конкретного студента

In [25]:
print('До')
query_2_13_ = f"""
            --sql   
            select * 
            from study_card
            where id_student = 6582 and id_subject = 10 and subject_method = 1
            --endsql
        """ 
make_query(query_2_13_)
with sq.connect('decanat.db') as con:
    query_2_13 = f"""
                --sql   
                update study_card
                set mark = 2
                where id_student = 6582 and id_subject = 10 and subject_method = 1
                --endsql
            """ 
    con.execute(query_2_13)
    

print('После')
make_query(query_2_13_)

До


,id,id_student,id_specialty,id_subject,subject_method,mark
0,1,6582,19,10,1,5


После


,id,id_student,id_specialty,id_subject,subject_method,mark
0,1,6582,19,10,1,2


#### 14. Обновить название специальности для всех студентов, где специальность имеет id = 2

In [26]:
with sq.connect('decanat.db') as con:
    query_2_14 = f"""
                --sql   
                update specialties
                set specialty_name = 'Новая специальность__'
                where id = 2
                --endsql
            """ 
    con.execute(query_2_14)
    

query_2_14_ = f"""
            --sql   
            select * 
            from specialties
            limit 7;
            --endsql
        """ 
make_query(query_2_14_)

,id,specialty_name,id_department
0,1,Архитектура вычислительных систем,1
1,2,Новая специальность__,1
2,3,Новая специальность,1
3,4,Математика и информатика,2
4,5,Информационные технологии,2
5,6,Веб-разработка и интернет-технологии,2
6,7,Физика и информатика,3


#### 15. Обновить все оценки студента с именем "Иван" на предмете "Математика" на значение 5

In [27]:
query_2_15_ = f"""
            --sql   
            select study_card.id, students.student_name, study_card.mark, subjects.subject_name, students.id
            from study_card
            left join students
            on students.id = study_card.id_student
            left join subjects
            on subjects.id = study_card.id_subject
            where students.student_name = 'Иван' and subjects.subject_name = 'Математика'
            limit 3;
            --endsql
        """ 
print('До')
make_query(query_2_15_)

with sq.connect('decanat.db') as con:
    query_2_15 = f"""
                --sql  
            update study_card
            set mark = 5
            where study_card.id = (select study_card.id as s_id
            from study_card
            left join students
            on students.id = study_card.id_student
            left join subjects
            on subjects.id = study_card.id_subject
            where students.student_name = 'Иван' and subjects.subject_name = 'Математика'
            limit 3)
                --endsql
            """ 
    con.execute(query_2_15)
    

print('После')
make_query(query_2_15_)

До


,id,student_name,mark,subject_name,id
0,10000,Иван,3,Математика,8252


После


,id,student_name,mark,subject_name,id
0,10000,Иван,5,Математика,8252


#### 16. Обновить название факультета на "Факультет информационных технологий" для всех кафедр, относящихся к этому факультету

In [28]:
with sq.connect('decanat.db') as con:
    query_2_16 = f"""
                --sql   
                update faculty
                set faculty_name = 'Новое название факультета информационных технологий'
                where faculty_name = 'Факультет Информационных Технологий'
                --endsql
            """ 
    con.execute(query_2_16)
    

query_2_16_ = f"""
            --sql   
            select *
            from faculty
            limit 7;
            --endsql
        """ 
make_query(query_2_16_)

,id,faculty_name
0,1,Новый факультет
1,2,Факультет Робототехники и Мехатроники
2,3,Факультет Энергетики и Энергомашиностроения
3,4,Факультет Нанотехнологий и Материаловедения
4,5,Факультет Авиационной и Космической Техники
5,6,Факультет Прикладной Математики и Информатики
6,7,Факультет Биотехнологии и Биоинженерии


#### 17. Обновить количество лабораторных часов на предмете "Физика" для специальности "Физика и информатика" на 30

In [29]:
with sq.connect('decanat.db') as con:
    query_2_17_ = f"""
            --sql   
            select specialties.specialty_name, subjects.subject_name, study_program.num_lab_hours
            from study_program
            left join specialties
            on specialties.id = study_program.id_specialtie
            left join subjects
            on subjects.id = study_program.id_subject
            where subjects.subject_name = 'Физика' and specialties.specialty_name = 'Физика и информатика'
            --endsql
        """ 
    make_query(query_2_17_)
    
    query_2_17 = f"""
                --sql   
                with table1 as (select study_program.id_specialtie, study_program.id_subject
            from study_program
            left join specialties
            on specialties.id = study_program.id_specialtie
            left join subjects
            on subjects.id = study_program.id_subject
            where subjects.subject_name = 'Физика' and specialties.specialty_name = 'Физика и информатика')
            
            
            update study_program
            set num_lab_hours = 30
            from table1
            where study_program.id_specialtie = table1.id_specialtie and study_program.id_subject = table1.id_subject
                --endsql
            """ 
    con.execute(query_2_17)
    


make_query(query_2_17_)

,specialty_name,subject_name,num_lab_hours
0,Физика и информатика,Физика,45


,specialty_name,subject_name,num_lab_hours
0,Физика и информатика,Физика,30


# Запросы на удаление из базы данных

#### 1. Удалить всех абитуриентов, у которых дата поступления ранее 2020 года.

In [30]:
print('До')
query_3_1_ = f"""
            --sql   
            select *
            from students
            order by entry_date
            limit 3;
            --endsql
        """ 
make_query(query_3_1_)

with sq.connect('decanat.db') as con:
    query_3_1 = f"""
                --sql  
                delete from students 
                WHERE  entry_date < '2020-01-01'
                --endsql
            """ 
    con.execute(query_3_1)
    

print('После')
make_query(query_3_1_)

До


,id,surname,student_name,fathers_name,gender,date_birth,address,phone,email,entry_date,student_specialty,group_num
0,4778,Чернев,Валентин,Сапронов,М,2000-08-20,ул. Озерковская 810,+7-(968)-998-78-44,aqua1809@outlook.com,2018-01-01,36,ИБА-2
1,6566,Терентьев,Ерофей,Старков,М,2006-05-12,Аллея Филевская 3-я 701,+7-(964)-713-18-69,singer2054@live.com,2018-01-01,73,ИС-17
2,7155,Жвалова,Феруза,Жвалевская,Ж,2005-02-27,Аллея Селезневская 90,+7-(969)-654-05-70,generic1894@duck.com,2018-01-01,69,ИВТ-14


После


,id,surname,student_name,fathers_name,gender,date_birth,address,phone,email,entry_date,student_specialty,group_num
0,3891,Сурова,Паула,Савкина,Ж,2004-08-16,Аллея Жидков 382,+7-(909)-407-56-22,wondering2006@gmail.com,2020-01-01,69,ИМТ-8
1,5874,Куликов,Александр,Гогуа,М,2000-03-07,Аллея Медынская 1348,+7-(968)-530-75-53,example2072@outlook.com,2020-01-01,58,ИБА-11
2,2297,Булочникова,Магда,Цветкова,Ж,2006-06-29,Аллея Алабушево 1163,+7-(927)-931-93-50,dictionaries2079@example.org,2020-01-02,88,ИБА-5


#### 2. Удалить все учебные планы, которые не связаны ни с одной специальностью.

In [31]:
with sq.connect('decanat.db') as con:
    query_3_2 = f"""
                --sql  
                delete from study_program 
                WHERE  id_specialtie is NULL
                --endsql
            """ 
    con.execute(query_3_2)
    
    query_3_2_ = f"""
                --sql   
                select *
                from study_program
                where id_specialtie is NULL
                limit 3;
                --endsql
            """ 
make_query(query_3_2_)

,id,id_specialtie,id_subject,id_subject_method,num_lecture_hours,num_practical_hours,num_lab_hours,final_task


#### 3. Удалить все кафедры, которые не относятся ни к одному факультету.

In [32]:
with sq.connect('decanat.db') as con:
    query_3_3 = f"""
                --sql  
                delete from departments 
                WHERE  id_faculty is NULL
                --endsql
            """ 
    con.execute(query_3_3)
    
    query_3_3_ = f"""
                --sql   
                select *
                from departments
                where id_faculty is NULL
                limit 3;
                --endsql
            """ 
make_query(query_3_3_)

,id,depatment_name,id_faculty


#### 4. Удалить все предметы, которые не входят ни в один учебный план.

In [33]:
with sq.connect('decanat.db') as con:
    print('До')
    query_3_4_ = f"""
                --sql   
                select subjects.subject_name, count(study_program.id) as num_programm
                from subjects
                left join study_program
                on study_program.id_subject = subjects.id
                group by subjects.subject_name
                having num_programm = 0
                --endsql
            """ 
            
    make_query(query_3_4_)
    query_3_4 = f"""
                --sql  
                with table1 as (
                    select subjects.subject_name, count(study_program.id) as num_programm
                from subjects
                left join study_program
                on study_program.id_subject = subjects.id
                group by subjects.subject_name
                having num_programm = 0
                )
                delete from subjects 
                WHERE subjects.subject_name in (select subject_name from table1)
                
                --endsql
                """
    con.execute(query_3_4)
    
print('После')
make_query(query_3_4_)

До


,subject_name,num_programm
0,Информационные технологии,0
1,Русский язык,0


После


,subject_name,num_programm


#### 5. Удалить всех студентов, не имеющих оценок.

In [34]:
with sq.connect('decanat.db') as con:
    query_3_5_ = f"""
                --sql   
                select students.id, study_card.mark
                from students
                left join study_card
                on study_card.id_student = students.id
                where study_card.mark is NULL
                --endsql
            """ 
    make_query(query_3_5_)
          
    query_3_5 = f"""
                --sql  
                delete from students
                WHERE  students.id in (select students.id
                from students
                left join study_card
                on study_card.id_student = students.id
                where study_card.mark is NULL)
                --endsql
            """ 
    con.execute(query_3_5)
    

make_query(query_3_5_)

,id,mark
0,4,None
1,25,None
2,27,None
3,29,None
4,30,None
...,...,...
2572,9986,None
2573,9987,None
2574,9993,None
2575,9995,None


,id,mark


#### 6. Удалить все записи из таблицы "Учебная карточка" для предмета "Математика" с формой сдачи "Экзамен"

In [35]:
with sq.connect('decanat.db') as con:
    query_3_6_ = f"""
                --sql   
                select *
                from study_card
                left join subjects
                on subjects.id = study_card.id_subject
                left join subject_methods
                on subject_methods.id = study_card.subject_method
                where subjects.subject_name = 'Математика' and subject_methods.subject_method_name = 'Экзамен'
                --endsql
            """ 
    make_query(query_3_6_)
          
    query_3_6 = f"""
                --sql  
                delete from study_card
                WHERE  study_card.id in (select study_card.id
                from study_card
                left join subjects
                on subjects.id = study_card.id_subject
                left join subject_methods
                on subject_methods.id = study_card.subject_method
                where subjects.subject_name = 'Математика' and subject_methods.subject_method_name = 'Экзамен')
                --endsql
            """ 
    con.execute(query_3_6)
    

make_query(query_3_6_)

,id,id_student,id_specialty,id_subject,subject_method,mark,id,subject_name,id,subject_method_name
0,10000,8252,70,12,1,5,12,Математика,1,Экзамен


,id,id_student,id_specialty,id_subject,subject_method,mark,id,subject_name,id,subject_method_name


#### 7. Удалить все записи из таблицы "Учебная карточка" для студентов, обучающихся на факультете "Биотехнологии и Биоинженерии"

In [36]:
with sq.connect('decanat.db') as con:
    query_3_7_ = f"""
                --sql   
                with table1 as (select study_card.id as s_id, specialties.id_department 
                from study_card
                left join specialties
                on specialties.id = study_card.id_specialty),
                
                table2 as (select * 
                from table1
                left join departments
                on departments.id = table1.id_department)
                
                select s_id, faculty.faculty_name
                from table2
                left join faculty
                on faculty.id = table2.id_faculty
                where faculty.faculty_name = 'Факультет Биотехнологии и Биоинженерии'
                --endsql
            """ 
    make_query(query_3_7_)
          
    query_3_7 = f"""
                --sql  
                delete from study_card
                WHERE  study_card.id in (
                    with table1 as (select study_card.id as s_id, specialties.id_department 
                from study_card
                left join specialties
                on specialties.id = study_card.id_specialty),
                
                table2 as (select * 
                from table1
                left join departments
                on departments.id = table1.id_department)
                
                select s_id
                from table2
                left join faculty
                on faculty.id = table2.id_faculty
                where faculty.faculty_name = 'Факультет Биотехнологии и Биоинженерии'
                )
                --endsql
            """ 
    con.execute(query_3_7)
    

make_query(query_3_7_)

,s_id,faculty_name
0,6,Факультет Биотехнологии и Биоинженерии
1,21,Факультет Биотехнологии и Биоинженерии
2,25,Факультет Биотехнологии и Биоинженерии
3,26,Факультет Биотехнологии и Биоинженерии
4,28,Факультет Биотехнологии и Биоинженерии
...,...,...
1060,9937,Факультет Биотехнологии и Биоинженерии
1061,9940,Факультет Биотехнологии и Биоинженерии
1062,9947,Факультет Биотехнологии и Биоинженерии
1063,9957,Факультет Биотехнологии и Биоинженерии


,s_id,faculty_name


#### 8. Удалить все записи из таблицы "Учебная карточка" для студентов, не сдавших ни одного экзамена

In [37]:
with sq.connect('decanat.db') as con:
    query_3_8_ = f"""
                --sql   
                select id_student, max(mark) as best_mark
                from study_card
                group by id_student
                having best_mark = 2
                
                --endsql
            """ 
    make_query(query_3_8_)
          
    query_3_8 = f"""
                --sql  
                delete from study_card
                where id_student in (
                    select id_student
                from study_card
                group by id_student
                having max(mark) = 2
                )
                --endsql
            """ 
    con.execute(query_3_8)
    

make_query(query_3_8_)

,id_student,best_mark
0,6,2
1,10,2
2,22,2
3,31,2
4,35,2
...,...,...
934,9899,2
935,9911,2
936,9915,2
937,9990,2


,id_student,best_mark


#### 9. Удалить всех абитуриентов, которые не поступили на заданную специальность 

P.S. Учитывались только студенты, обучающиеся на специальности

In [38]:
with sq.connect('decanat.db') as con:
    query_3_9_ = f"""
                --sql   
                select *
                from students
                where entry_date is NULL
                
                --endsql
            """ 
    make_query(query_3_9_)
          
    query_3_9 = f"""
                --sql  
                delete from students
                where id in (
                    select id
                from students
                where entry_date is NULL
                )
                --endsql
            """ 
    con.execute(query_3_9)
    

make_query(query_3_9_)

,id,surname,student_name,fathers_name,gender,date_birth,address,phone,email,entry_date,student_specialty,group_num


,id,surname,student_name,fathers_name,gender,date_birth,address,phone,email,entry_date,student_specialty,group_num


#### 10. Удалить все учебные карточки, связанные с предметом, который больше не входит в учебный план по заданной специальности

In [39]:
with sq.connect('decanat.db') as con:
    query_3_10_ = f"""
                --sql   
                select *
                from study_card
                where id_specialty = 13 and id_subject = 1
                
                --endsql
            """ 
    make_query(query_3_10_)
          
    query_3_10 = f"""
                --sql  
                delete from study_card
                where id in (
                    select id
                from study_card
                where id_specialty = 13 and id_subject = 1
                )
                --endsql
            """ 
    con.execute(query_3_10)
    

make_query(query_3_10_)

,id,id_student,id_specialty,id_subject,subject_method,mark
0,950,6215,13,1,2,3
1,2044,3563,13,1,3,4
2,2087,3445,13,1,1,4
3,2293,6677,13,1,4,3
4,3538,4494,13,1,2,5
5,3822,2100,13,1,3,4
6,4139,9557,13,1,1,5
7,4307,4603,13,1,1,2
8,6389,9366,13,1,2,2
9,7432,6677,13,1,4,4


,id,id_student,id_specialty,id_subject,subject_method,mark


#### 11. Удалить всех абитуриентов, не сдавших заданный предмет

In [40]:
with sq.connect('decanat.db') as con:
    query_3_11_ = f"""
                --sql   
                select id_student, mark, subjects.subject_name
                from study_card
                left join subjects
                on subjects.id = study_card.id_subject
                where subjects.subject_name = 'Психология общения' and study_card.mark = 2
                --endsql
            """ 
    make_query(query_3_11_)
          
    query_3_11 = f"""
                --sql  
                delete from students
                where id in (
                    select id_student
                from study_card
                left join subjects
                on subjects.id = study_card.id_subject
                where subjects.subject_name = 'Психология общения' and study_card.mark = 2
                )
                --endsql
            """ 
    con.execute(query_3_11)
    

,id_student,mark,subject_name
0,4566,2,Психология общения
1,422,2,Психология общения
2,3011,2,Психология общения
3,5751,2,Психология общения
4,9064,2,Психология общения
...,...,...,...
103,3638,2,Психология общения
104,2932,2,Психология общения
105,9359,2,Психология общения
106,9049,2,Психология общения


#### 12. Удалить все учебные карточки, связанные с заданным преподавателем

В базе данных нет информации о преподавателях

#### 13. Удалить все учебные карточки, связанные с заданной формой сдачи предмета

In [41]:
with sq.connect('decanat.db') as con:
    query_3_13_ = f"""
                --sql   
                select id
                from study_card
                where subject_method = 4
                --endsql
            """ 
    make_query(query_3_13_)
          
    query_3_13 = f"""
                --sql  
                delete from study_card
                where id in (
                    select id
                from study_card
                where subject_method = 4
                )
                --endsql
            """ 
    con.execute(query_3_13)
    
make_query(query_3_13_)

,id
0,3
1,7
2,9
3,17
4,19
...,...
1956,9994
1957,9995
1958,9996
1959,9997


,id


#### 14. Удалить всех абитуриентов, не зачисленных на заданную специальность

P.S. Такой запрос большую часть студентов. Из 10_000 человек останется около 100. Поэтому запрос на удаление составлен, но закомментирован. Иначе запросы 15, 16, 17 потеряют смысл

In [42]:
with sq.connect('decanat.db') as con:
    query_3_14_ = f"""
                --sql   
                select students.id, surname, student_name, specialty_name
                from students
                left join specialties
                on specialties.id = students.student_specialty
                where specialties.specialty_name != 'Когнитивные вычисления'
                
            
                --endsql
            """ 
    make_query(query_3_14_)
          
    query_3_14 = f"""
                --sql  
                delete from students
                where id in (
                    select students.id
                from students
                left join specialties
                on specialties.id = students.student_specialty
                where specialties.specialty_name != 'Когнитивные вычисления'
                )
                
                --endsql
            """ 
    # con.execute(query_3_14)
    

,id,surname,student_name,specialty_name
0,2,Остратов,Валентин,Конструкция и эксплуатация автомобилей
1,3,Кириллова,Мирослава,Численные методы
2,5,Обухова,Ея,Биомедицинская визуализация
3,6,Евстафьев,Герман,Системы управления воздушным движением
4,8,Тарн,Назар,Космические двигатели и тяга
...,...,...,...,...
4334,9981,Зыкова,Милен,Системы автоматизированного проектирования
4335,9983,Гермогенова,Мина,Математическое моделирование
4336,9984,Казимиров,Пантелеймон,Математика и информатика
4337,9994,Балков,Добрыня,Промышленная биотехнология


#### 15. Удалить всех абитуриентов, которые подавали заявку на специальность с названием "Физика и информатика", вместе с их учебными карточками

In [43]:
with sq.connect('decanat.db') as con:
    query_3_15_ = f"""
                --sql   
                select study_card.id as s_id, id_student
                from study_card
                left join specialties
                on specialties.id = study_card.id_specialty
                where specialty_name = 'Физика и информатика'
                --endsql
            """ 
    make_query(query_3_15_)
          
    query_3_15 = f"""
                --sql  
                delete from study_card
                where id in (
                    select study_card.id
                from study_card
                left join specialties
                on specialties.id = study_card.id_specialty
                where specialty_name = 'Физика и информатика'
                )
                
                --endsql
            """ 
    con.execute(query_3_15)
    
    query_3_15 = f"""
                --sql  
                delete from students
                where id in (
                    select id_student
                from study_card
                left join specialties
                on specialties.id = study_card.id_specialty
                where specialty_name = 'Физика и информатика'
                )
                --endsql
            """ 
            
    con.execute(query_3_15)
    
make_query(query_3_15_)

,s_id,id_student
0,57,7419
1,153,4110
2,190,4372
3,331,9120
4,351,7181
...,...,...
58,8893,40
59,9082,9093
60,9371,3523
61,9455,1780


,s_id,id_student


#### 16. Удалить всех студентов, которые учатся на специальности "Информационные технологии" и сдали экзамен по предмету "Алгоритмы и структуры данных" на более низкую оценку, вместе с их учебными карточками

In [44]:
with sq.connect('decanat.db') as con:
    query_3_16_ = f"""
                --sql   
                select study_card.id as s_id, id_student
                from study_card
                --endsql
            """ 
    make_query(query_3_16_)

    query_3_15 = f"""
                --sql  
                delete from students
                where id in (
                   
                )
                --endsql
            """ 
            
    #con.execute(query_3_16)
    
make_query(query_3_16_)

,s_id,id_student
0,4,7843
1,5,1613
2,8,7516
3,11,2377
4,12,381
...,...,...
5833,9987,5599
5834,9988,729
5835,9989,5542
5836,9992,8707


,s_id,id_student
0,4,7843
1,5,1613
2,8,7516
3,11,2377
4,12,381
...,...,...
5833,9987,5599
5834,9988,729
5835,9989,5542
5836,9992,8707


#### 17. Удалить всех студентов, которые не сдали экзамен по предмету "История" в установленный срок, вместе с их учебными карточками

In [45]:
with sq.connect('decanat.db') as con:
    query_3_17_ = f"""
                --sql   
                select *
                from study_card
                left join subjects
                on subjects.id = study_card.id_subject
                where subjects.subject_name = 'История' and mark = 2
                --endsql
            """ 
    make_query(query_3_17_)

    query_3_17 = f"""
                --sql  
                delete from students
                where id in (
                   select study_card.id_student
                from study_card
                left join subjects
                on subjects.id = study_card.id_subject
                where subjects.subject_name = 'История' and mark = 2
                )
                --endsql
            """ 
            
    con.execute(query_3_17)
    
    
    query_3_17 = f"""
                --sql  
                delete from study_card
                where id in (
                   select study_card.id
                from study_card
                left join subjects
                on subjects.id = study_card.id_subject
                where subjects.subject_name = 'История' and mark = 2
                )
                --endsql
            """ 
            
    con.execute(query_3_17)
    
make_query(query_3_17_)

,id,id_student,id_specialty,id_subject,subject_method,mark,id,subject_name
0,58,8226,49,10,1,2,10,История
1,92,9164,50,10,3,2,10,История
2,94,2356,74,10,2,2,10,История
3,294,1671,64,10,1,2,10,История
4,338,2012,38,10,3,2,10,История
...,...,...,...,...,...,...,...,...
87,8919,6748,18,10,2,2,10,История
88,9011,965,12,10,1,2,10,История
89,9661,1134,87,10,2,2,10,История
90,9862,6802,34,10,2,2,10,История


,id,id_student,id_specialty,id_subject,subject_method,mark,id,subject_name
